# Mapeo campos 

|Fuente | Sources |
|---|---|
|Nombre base para flora de Bogota | Taxa |
|Comentario de validación | Taxa.Comment |
|Para revisión | Taxa.CheckPriority 0 or 1 |
|Endemica | Taxa.Distribution |
|Estado de amenaza | RedListAssessments with Standard='JBB'
|Origen y status | Origins |
|Trabajo de No nativas Flora de Bogotá | Origins.Type???
|Determinador en fuente | Identifications.IdentifiedBy Merge with next field |
|Determinador Depurado | Identifications.IdentifiedBy |
|Calificador en fuente | Identifications.Certainty |
|Fecha_Determinacion en fuente | Identifications.Date |
|Fecha_Determinacion_(Year) | Identifications.Date |
|Fecha_Determinacion_(Month) | Identifications.Date |
|Fecha_Determinacion_(Day) | Identifications.Date |
|State | Areas |
|County | Areas |
|Localidad | Locations |
|Código de localidad georreferenciada | Locations.BD2020ID |
|Latitud_original | Locations.Latitude |
|Longitud_original | Locations.Longitud |
|Elev__minima | Locations.ElevationMin |
|Elev__maxima | Locations.ElevationMax |
|Latitud_decimal | Geocodings.InterpretedLat, Geocoder is DB_2020 |
|Longitud_decimal | Geocodings.InterpretedLon, Geocoder is DB_2020 |
|Latitud para trabajo de mapas | Geocodings.InterpretedLat, Geocoder is mapas_2020 |
|Longitud para trabajo de mapas | Geocodings.InterpretedLon, Geocoder is mapas_2020 |
|Extensión de georreferenciación | Geocodins.Uncertainty ???
|DATUM | Geocodings.Datum |
|Colector depurado | Occurrences.Collector |
|Colector principal | If no colector depurado, Occurrences.Collector, otherwise Occurrences.CollectorVerbatim  |
|Número de colector | Occurrences.CollectionNumber & Occurrences.CollectionNumberVerbatim |
|Fecha_Inicial_(Year) | Occurrences.DateInit, Occurrences.DateEnd |
|Fecha_Inicial_(Month) | Occurrences.DateInit, Occurrences.DateEnd |
|Fecha_Inicial_(day) | Occurrences.DateInit, Occurrences.DateEnd |
|Notas_Localidad_y_Habitat | Occurrences.Comment  |
|Fenologia | Occurrences.PhenoState |
|Descripcion ejemplar | Occurrences.Comment |
|Usos | Occurrences.Use |
|Nombres_comunes | Occurrences.CommonName |
|Comentarios generales | Occurrences.Comment |
|Referencia bibligráfica | Occurrence.Sources |
|Numero original en Base 2015 | Occurrence.DB2015ID |
|Origen para herbario | Specimens.Institution  |
|Número de identificación | Specimens.SpecimenCode |
|Fecha_Inicial | REMOVE |
|Notas de altura | REMOVE ???????
|Familia Revisada | REMOVE |
|Género revisado | REMOVE |
|Comentarios de determinación en fuente | REMOVE |
|Nivel taxonómico | REMOVE, info retrieved from TPRS |
|Familia en fuente | REMOVE  |
|Género en fuente | REMOVE  |
|Epíteto específico en fuente | REMOVE  |
|Autor del epíteto específico en fuente | REMOVE  |
|Epíteto infraespecífico | REMOVE  |
|Autor del infraespecífico en fuente | REMOVE  |
|Nombre científico en fuente | REMOVE  |
|Nombre validado | REMOVE  |
|Nivel taxonómico en fuente | REMOVE  |
|Herbario de proveniencia | REMOVE  |

In [3]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np

import requests
import json
import re
import time
from datetime import datetime

import pykew.powo as powo
from pygbif import species, registry

import credentials

In [25]:
orifile = '../Datos/Flora_de_Bogota/Registros_2020/FlBogota2020.csv'
ori = pd.read_csv(orifile, dtype={'Elev__minima': str, 'Elev__maxima': str})

In [ ]:
tnrs_url = 'https://tnrsapi.xyz/tnrs_api.php'

In [ ]:
def tropicos(name, api_key):
	service = 'http://services.tropicos.org/name'
	r = requests.get(
		f'{service}/search/?apikey={api_key}&name={name}&format=json'
	)
	#s = requests.get(
	#	f'{service}/{r['NameId']}/highertaxa/?apikey={api_key}&name={name}&format=json'
	#)
	
	return r.json()


In [ ]:
def powo_query(name):
	act_name = ''
	bits = re.split(r'\s+', name)

	if bits[0] == '×' or bits[1] == '×':
		act_name = ' '.join(bits[:3])
	else:
		act_name = ' '.join(bits[:2])
	
	r = powo.search(act_name)

	if r.size() == 1:
		results = [i for i in r]
		return (results[0]['name'], results[0]['author'], results[0]['rank'])
	
	elif r.size() > 1:
		return f'Multiples results with name {name}'
	
	else:
		return f'No results with name {name}'

### Si la consulta a TNRS ya fue realizada y los resultados guardados:

In [ ]:
# Map between original name and corrected version by TNRS
taxa_map_file = 'tnrs_query.csv'
taxa_map = pd.read_csv(taxa_map_file)

### De lo contrario, aqui se realiza la consulta

In [ ]:
orinames = ori['Nombre base para flora de Bogota'].unique()
query = [[i, j] for i,j in enumerate(orinames) if pd.notna(j)]

r = requests.post(
	tnrs_url, 
	headers={"Content-Type": "application/json"},
	data=json.dumps({
		'data': query,
		'opts': {
			'sources': ['wcvp','wfo'],
			'class': 'wcvp',
			'mode': 'resolve',
			'matches' : 'best'
		}		
	})	
)

In [ ]:
taxa_map = pd.DataFrame.from_records(results)
taxa_map.to_csv('tnrs_query.csv', index=False)

In [ ]:
taxa_map.loc[taxa_map.Overall_score == '', 'Overall_score'] = 'NAN'
taxa_map['Overall_score'] = taxa_map.Overall_score.astype(np.float32)

# Aqui se continua la normalizacion de nombres

### Consulta a POWO de hibridos

Primero remover hibridos no publicados, solo se registran a nivel de determinacion

In [ ]:
taxa_map = taxa_map.drop(
	index=taxa_map.loc[taxa_map.Name_submitted.str.contains(r'Salvia [\w\.\s]+ ×')].index
)

In [ ]:
for row in taxa_map.loc[taxa_map.Name_submitted.str.contains('×')].itertuples():
	#print(row.Name_submitted)
	po = powo_query(row.Name_submitted)
	if isinstance(po, tuple):
		taxa_map.loc[row.Index, 'Name_matched'] = po[0]
		taxa_map.loc[row.Index, 'Author_matched'] = po[1]
		taxa_map.loc[row.Index, 'Name_matched_rank'] = po[2].lower()
	else:
		print(po)
	time.sleep(1)

In [ ]:
# Manually correct names couldn't be matched easily by POWO,
# but there is a simple solution there 

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Pelargonium × hortorum')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Pelargonium × hortorum', 'L.H.Bailey', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Abelia × grandiflora')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Abelia × grandiflora', '(Rovelli ex André) Rehder', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Lavatera × clementii')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Lavatera × clementii', 'Cheek', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Polygala × dalmasiana')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Polygala', np.nan, 'genus')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Epidendrum chicoense')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Epidendrum', np.nan, 'genus')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Plantae')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = (np.nan, np.nan, np.nan)

# TNRS couldn't get appropriate response for the following

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Callianthe striata')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Callianthe striata', '(G.F.Dicks. ex Lindl.) Donnell', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Phlegmariurus acifolius')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Phlegmariurus acifolius', '(Rolleri) B.Øllg.', 'species')


In [ ]:
# Save eliminated binomials from the database, they probably are invalid or unpublished names
taxa_map.loc[(taxa_map.Overall_score < 0.9) & (taxa_map.Name_matched_rank == 'genus'),
	['Name_submitted', 'Name_matched', 'Name_matched_rank']
	].to_csv('binomios_eliminados.csv', index=False)

In [ ]:
# Remove lichens: Ramalina, Usnea, Sticta, Candelariella
# Remove general indets: 'Pteridophyta', 'Magnoliophyta', 'Plantae', 'Tracheophyta'

taxa_map = taxa_map.drop(
	index=taxa_map[taxa_map.Name_submitted.isin([
		'Ramalina', 'Usnea', 'Sticta', 'Candelariella',
		'Pteridophyta', 'Magnoliophyta', 'Plantae', 'Tracheophyta'
	])].index
)

In [ ]:
taxa_map.to_csv('taxa_map.csv', index=False)

### Obtener jerarquía taxonómica de World Flora Online

In [5]:
taxa_map = pd.read_csv('taxa_map.csv')

In [6]:
wfo_file = '../../Botanical_databases/world_flora_online/classification.csv'

In [7]:
datyp = {'localID':str, 'subfamily': str, 'tribe': str, 'subtribe': str, 
 	'subgenus':str, 'infraspecificEpithet': str, 'tplID': str}

In [8]:
wfo = pd.read_csv(wfo_file, sep='\t', encoding='latin', dtype=datyp)

In [9]:
taxa_in = taxa_map.groupby(['Name_matched','Author_matched']
	).size(
	).reset_index(
	).drop(columns=0
	).rename(columns={'Name_matched': 'Name', 'Author_matched':'Author'})

In [10]:
# For some reason Passiflora tripartita is retrieved twice, one with the wrong authorship
taxa_in = taxa_in.drop(
	index=taxa_in[
		(taxa_in.Name == 'Passiflora tripartita') & 
		(taxa_in.Author == '(Juss.) Poir.')
	].index).reset_index()

In [11]:
def get_family(name):
	ret = np.nan
	if pd.isna(name): return np.nan
	bits = re.split(r'\s+', name)
	nname = bits[0]
	out = wfo.loc[(wfo.scientificName == nname) & (wfo.taxonRank == 'genus')]

	if out.shape[0] == 1: 
		ret =  out['family'].item()
	
	elif out.shape[0] > 1:

		if out[out['nomenclaturalStatus'] == 'Valid'].shape[0] == 1:
			ret =  out.loc[out['nomenclaturalStatus'] == 'Valid', 'family'].item()

		elif out[out['taxonomicStatus'] == 'Accepted'].shape[0] == 1:
			ret =  out.loc[out['taxonomicStatus'] == 'Accepted', 'family'].item()

		else:
			if len(bits) > 1: nname = f'{bits[0]} {bits[1]}'
			out = wfo.loc[(wfo.scientificName == nname)]
			fams = out.family.unique()
			fams = [x for x in fams if pd.notna(x)]

			if len(fams) == 1:
				ret =  fams[0]		
		
	elif out.shape[0] < 1 and re.search(r'\w+aceae$', nname):
		out = wfo.loc[(wfo.scientificName == nname) & (wfo.taxonRank == 'family')]
		fams = out.family.unique()
		fams = [x for x in fams if pd.notna(x)]

		if len(fams) == 1:
			ret =  fams[0]

	return ret


In [12]:
taxa_in['wfo_family'] = taxa_in.Name.apply(get_family)

In [13]:
# Some missing names in World Flora Online
taxa_in.loc[taxa_in.Name == 'Hydrophyllaceae', 'wfo_family'] = 'Hydrophyllaceae'
taxa_in.loc[taxa_in.Name == 'Stemmatella', 'wfo_family'] = 'Asteraceae'
taxa_in.loc[taxa_in.Name == 'Coniza', 'wfo_family'] = 'Asteraceae'
taxa_in.loc[taxa_in.Name == '× Agropogon lutosus', 'wfo_family'] = 'Poaceae'


In [14]:
famis = taxa_in.groupby('wfo_family'
	).size(
	).reset_index(
	)

In [15]:
def get_papa(name):
	ret = np.nan

	if pd.notna(name):
		out = wfo.loc[(wfo.scientificName == name), 'parentNameUsageID']
		th = [x for x in out if pd.notna(x)]
		
		if len(th) == 1:
			parid = th[0]
			ret = wfo.loc[(wfo.taxonID == parid), 'scientificName'].item()
	
	return ret

In [16]:
famis['wfo_order'] = famis.wfo_family.apply(get_papa)

In [17]:
famis.loc[famis.wfo_family == 'Hydrophyllaceae', 'wfo_order'] = 'Solanales'

In [18]:
famis['wfo_clade'] = famis.wfo_order.apply(get_papa)

In [19]:
taxa_in = taxa_in.merge(famis, 'left', 'wfo_family')

In [20]:
taxa_in.drop(columns=0).to_csv('taxa_in.csv', index=False)

### Verificación de nombres realizada y previamente guardada localmente

Abajo se formalizan las jerarquías dado el diseño de la tabla Taxa en la DB

In [ ]:
taxa_in = pd.read_csv('taxa_in.csv')

In [22]:
taxa_in = taxa_in.reset_index(drop=True)

In [23]:
taxa_in['TaxonID'] = taxa_in.index + 1

In [21]:
taxa_in[['CheckPriority', 'Distribution', 'Parent']] = np.nan, np.nan, np.nan

In [26]:
oriname = 'Nombre base para flora de Bogota'

for row in taxa_in.itertuples():
	
	qu = (ori[oriname] == row.Name) & (ori.Endemica == 'SI')
	
	if ori.loc[qu].shape[0] > 0:
		taxa_in.loc[taxa_in.Name == row.Name, 'Distribution'] = 0

	qu = (ori[oriname] == row.Name) & (ori['Para revisión'] == 'SI')
	
	if ori.loc[qu].shape[0] > 0:
		taxa_in.loc[taxa_in.Name == row.Name, 'CheckPriority'] = 5

In [29]:
last_id = taxa_in.TaxonID.iloc[-1]

In [32]:
for th in taxa_in.wfo_clade.unique():
	qu = taxa_in.loc[taxa_in.Name == th]
	if qu.shape[0] == 0:
		last_id += 1
		rec = {i:np.nan for i in taxa_in.columns}
		rec['Name'] = th
		rec['TaxonID'] = last_id
		taxa_in = pd.concat([
			taxa_in, 
			pd.DataFrame.from_records([rec])
		])


In [34]:
for th in taxa_in.wfo_order[taxa_in.wfo_order.notna()].unique():
	qu = taxa_in.loc[taxa_in.Name == th]
	if qu.shape[0] == 0:
		last_id += 1
		papa = taxa_in.loc[taxa_in.wfo_order == th, 'wfo_clade'].iloc[0]
		papaid = taxa_in.loc[taxa_in.Name == papa, 'TaxonID'].iloc[0]
		rec = {i:np.nan for i in taxa_in.columns}
		rec['Name'] = th
		rec['Parent'] = papaid
		rec['TaxonID'] = last_id
		taxa_in = pd.concat([
			taxa_in, 
			pd.DataFrame.from_records([rec])
		])


In [41]:
for th in taxa_in.wfo_family[taxa_in.wfo_family.notna()].unique():
	qu = taxa_in.loc[taxa_in.Name == th]
	if qu.shape[0] == 0:
		last_id += 1
		papa = taxa_in.loc[taxa_in.wfo_family == th, 'wfo_order'].iloc[0]
		papaid = taxa_in.loc[taxa_in.Name == papa, 'TaxonID'].iloc[0]
		rec = {i:np.nan for i in taxa_in.columns}
		rec['Name'] = th
		rec['Parent'] = papaid
		rec['TaxonID'] = last_id
		taxa_in = pd.concat([
			taxa_in, 
			pd.DataFrame.from_records([rec])
		])
	else:
		papa = taxa_in.loc[taxa_in.wfo_family == th, 'wfo_order'].iloc[0]
		papaid = taxa_in.loc[taxa_in.Name == papa, 'TaxonID'].iloc[0]
		taxa_in.loc[taxa_in.Name == th, 'Parent'] = papaid



In [49]:
no_gens = taxa_in.wfo_family.unique().tolist() + \
	taxa_in.wfo_order.unique().tolist() + \
	taxa_in.wfo_clade.unique().tolist() + ['×']

no_gens = [i for i in no_gens if pd.notna(i)]

In [50]:
gens = [x for x in taxa_in.Name.apply(lambda x: re.split(r'\s+', x)[0]).unique()]
gens = [x for x in gens if not x in no_gens]

In [51]:
for gn in gens:
	#print(gn)
	
	if pd.notna(gn):

		qu = taxa_in.loc[taxa_in.Name == gn]

		if qu.shape[0] == 0:
			last_id += 1
			papa = taxa_in.loc[taxa_in.Name.str.startswith(f'{gn} '), 'wfo_family'].iloc[0]
			#print(papa)
			papaid = taxa_in.loc[taxa_in.Name == papa, 'TaxonID'].iloc[0]
			#print(papaid)
			rec = {i:np.nan for i in taxa_in.columns}
			rec['Name'] = gn
			rec['Parent'] = papaid
			rec['TaxonID'] = last_id
			taxa_in = pd.concat([
				taxa_in, 
				pd.DataFrame.from_records([rec])
			])

		else:
			papa = taxa_in.loc[taxa_in.Name == gn, 'wfo_family'].iloc[0]
			#print(papa)
			papaid = taxa_in.loc[taxa_in.Name == papa, 'TaxonID'].iloc[0]
			#print(papaid)
			taxa_in.loc[taxa_in.Name == gn, 'Parent'] = papaid



Por alguna razón Vriesea está incorrectamente clasificada

In [72]:
taxa_in.loc[taxa_in.Name == 'Bromeliaceae']

,index,Name,Author,wfo_family,0,wfo_order,wfo_clade,CheckPriority,Distribution,Parent,TaxonID
0,NaN,Bromeliaceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3166.0,3230


In [73]:
taxa_in.loc[taxa_in.Name == 'Vriesea', 'Parent'] = 3230

In [76]:
for sp in taxa_in.loc[
	taxa_in.Name.str.contains(r'^[\w\-]+\s+\w+') |
	taxa_in.Name.str.contains(r'\w+\s+×'),
	'Name'
	].unique():

	#print(sp)
	gn = re.split(r'\s+', sp)[0]
	#print(gn)
	qu = taxa_in.loc[taxa_in.Name == gn]
	papaid = taxa_in.loc[taxa_in.Name == gn, 'TaxonID'].iloc[0]
	#print(papaid)
	taxa_in.loc[taxa_in.Name == sp, 'Parent'] = papaid

In [80]:
taxa_in.loc[taxa_in.Name == '× Agropogon lutosus', 'Parent'
	] = taxa_in.loc[taxa_in.Name == 'Andropogon'].index[0]

In [82]:
taxa_in[['AcceptedName', 'Authority', 'Protologue', 'Comment']
	] = np.nan, np.nan, np.nan, np.nan

In [84]:
taxa_in[['TaxonID', 'Name', 'Author', 'CheckPriority', 'Distribution', 'Parent',
	'AcceptedName', 'Authority', 'Protologue', 'Comment']].to_csv(
	'taxa_in.csv', index=False)

# Insert Taxa table

If all nomenclatural and classification validation has been conducted, read `taxa_in.csv` file and directly do insertion.

In [2]:
taxa_in = pd.read_csv('taxa_in.csv')

In [13]:
conn_str = 'mysql+mysqlconnector://' + \
	f'{credentials.mysql['username']}:{credentials.mysql['password']}' + \
	'@localhost:3306/Mutis'
engine = create_engine(conn_str)
connection = engine.connect()

In [ ]:
#connection.execute(text('SET FOREIGN_KEY_CHECKS = 0'))

In [14]:
taxa_in[['TaxonID', 'Name', 'Author', 'CheckPriority', 'Distribution', 'Parent',
	'AcceptedName', 'Authority', 'Protologue', 'Comment']
	].to_sql('Taxa', engine, if_exists='append', index=False, chunksize=10000, method='multi')

-1

# Validation personas y grupos de personas 

Normalización de nombres de personas y de grupos para inserción en tablas Persons, PeoplePersons y People

In [63]:
from rapidfuzz import fuzz, utils, distance

In [4]:
orifile = '../Datos/Flora_de_Bogota/Registros_2020/FlBogota2020.csv'

colpp = ['Determinador en fuente', 'Determinador Depurado', 'Colector depurado', 
 'Colector principal', 'Referencia bibligráfica']

ori = pd.read_csv(orifile, dtype={'Elev__minima': str, 'Elev__maxima': str},
	usecols=colpp)

In [5]:
ori = ori.rename(columns={
	'Determinador en fuente':'detfu', 
	'Determinador Depurado':'detde', 
	'Colector depurado': 'colde', 
	'Colector principal': 'colpri',
	'Referencia bibligráfica': 'ref'
})

In [7]:
ori[ori.detfu.notna()].shape, ori[ori.detde.notna()].shape, \
	ori[ori.colde.notna()].shape, ori[ori.colpri.notna()].shape

((26982, 5), (42024, 5), (42177, 5), (38071, 5))

In [275]:
parties = ori.groupby('colde'
	).size(
	).reset_index(
	).sort_values('colde'
	).rename(columns={'colde':'party'}
	).drop(columns=0)

parties['person'] = parties.party

In [276]:
dets = ori.groupby('detde'
	).size(
	).reset_index(
	).sort_values('detde'
	).rename(columns={'detde':'party'}
	).drop(columns=0)

dets[['person0', 'person1']] = dets.party.str.split(r'\s+[&y]\s+',expand=True)

In [277]:
dets = dets.melt(id_vars='party', value_vars=['person0', 'person1'], 
	value_name='person').sort_values('party'
	).drop(columns='variable')

dets.loc[dets.person.isna(), 'person'] = dets.loc[dets.person.isna(), 'party']

In [ ]:
#parties = pd.concat([parties, dets])

In [ ]:
#parties = parties.groupby(['party', 'person'], dropna=False
#	).size(
#	).reset_index(
#	).drop(columns=0
#	).reset_index(drop=True)

In [278]:
jper = parties.groupby('person').size().reset_index().drop(columns=0)

In [279]:
def sim_scores(a_str):
	jper['jw'] = jper.person.apply(
		lambda x: distance.JaroWinkler.similarity(a_str, x)
	)
	jper['part'] = jper.person.apply(
		lambda x: fuzz.partial_ratio(a_str, x)
	)
	jper['ratio'] = jper.person.apply(
		lambda x: fuzz.ratio(a_str, x)
	)

	out = (
		jper.loc[(jper.jw == jper.jw.max()), 'person'].iloc[0],
		jper.jw.max(),
		jper.loc[(jper.part == jper.part.max()), 'person'].iloc[0],
		jper.part.max(),
		jper.loc[(jper.ratio == jper.ratio.max()), 'person'].iloc[0],
		jper.ratio.max()
	)

	return out

In [280]:
dets[['jw', 'jw_sc', 'part', 'part_sc', 'ratio', 'ratio_sc']] = dets.apply(
	lambda x: sim_scores(x.person), axis=1, result_type='expand'
)

In [298]:
val = 94
dets.loc[(dets.ratio_sc > val) & (dets.ratio_sc < val + 1), ['person', 'ratio', 'ratio_sc']].head(50)

,person,ratio,ratio_sc
340,"Fernández-A., J.L","Fernández-A., J. L.",94.444444
1509,"Fernández-A., J.L","Fernández-A., J. L.",94.444444


In [314]:
val = 95
dets.loc[(dets.part_sc > val) & (dets.part_sc < val + 1), ['person', 'part', 'part_sc']].head(50)

,person,part,part_sc
1372,"Castro, Sa.","Castro, S.L.",95.238095
203,"Castro, Sa.","Castro, S.L.",95.238095
227,"Cogollo, A.","Cogollo, Angélica",95.238095
1396,"Cogollo, A.","Cogollo, Angélica",95.238095
1578,"Giraldo, C.","Giraldo, Catalina",95.238095
409,"Giraldo, C.","Giraldo, Catalina",95.238095
509,"Hourteig, A.","Lourteig, A.",95.652174
1678,"Hourteig, A.","Lourteig, A.",95.652174
626,"Mahecha, Gilberto","Mahecha, G.",95.238095
1795,"Mahecha, Gilberto","Mahecha, G.",95.238095


In [308]:
val = 0.98
dets.loc[(dets.jw_sc > val) & (dets.jw_sc < val + 0.01), ['person', 'jw', 'jw_sc']].head(50)

,person,jw,jw_sc
1372,"Castro, Sa.","Castro, S.",0.981818
203,"Castro, Sa.","Castro, S.",0.981818
1419,"Cortés-S., S. P.","Cortés-S., S.P.",0.987500
250,"Cortés-S., S. P.","Cortés-S., S.P.",0.987500
1524,"Forero, Di.","Forero, D.",0.981818
355,"Forero, Di.","Forero, D.",0.981818
454,"Gutiérrez-G., V.","Gutiérrez-V., G.",0.987500
1623,"Gutiérrez-G., V.","Gutiérrez-V., G.",0.987500
1628,"Gómez, J. C.","Gómez, J.C.",0.983333
459,"Gómez, J. C.","Gómez, J.C.",0.983333


In [22]:
colpri_map = ori.loc[ori.colde.isna()].groupby('colpri').size().reset_index()

In [64]:
fuzz.partial_ratio('hola pepe', 'hoLa perro', processor=utils.default_process)

87.5

In [65]:
fuzz.ratio('hola pepe', 'hoLa perro', processor=utils.default_process)

73.6842105263158

In [67]:
distance.JaroWinkler.similarity('hola pepe', 'hoLa perro', processor=utils.default_process)

0.8955555555555555

In [23]:
colpri_map

,colpri,0
0,"Baptiste, H.",1
1,"Cabrera, Diego",7
2,"Cabrera-Amaya, Diego Mauricio",2
3,"Calbi, M.",2
4,"Camacho, I.",15
5,"Camacho, Shirley",2
6,"Cortés-Ballén, Lorena Andrea",3
7,"Fajardo, F.",80
8,"Fajardo, Francisco",30
9,"Fajardo-G., F.",60


In [ ]:
ori.loc[ori.colde.isna() & ori.colpri.notna()].groupby('colpri').size()

colpri
Baptiste, H.                       1
Cabrera, Diego                     7
Cabrera-Amaya, Diego Mauricio      2
Calbi, M.                          2
Camacho, I.                       15
Camacho, Shirley                   2
Cortés-Ballén, Lorena Andrea       3
Fajardo, F.                       80
Fajardo, Francisco                30
Fajardo-G., F.                    60
Fajardo-Gutiérrez, Francisco       2
Jara, A.                           1
Jara, Adolfo                       8
Lopera, María Catalina             2
Lopera-Doncel, María Catalina      1
López, José                        1
López-Cruz, José                   5
López-O., M.                       1
Martín, R.                        10
Martínez-Peña, Myriam Liliana      5
Marín, César                      18
Medellín-Z., D.                    3
Medellín-Z., Diana                 9
Montoya, Angela                    1
Montoya-Q., Angela                26
Morales-Rozo, Clara                1
Muñoz, José                    

In [22]:
ori.groupby(['Determinador en fuente', 'Determinador Depurado']).size()

Determinador en fuente  Determinador Depurado
!A. Alvarez (NY),       Álvarez, A.                1
!D. Grimm (CAS),        Grimm, D.                  1
!J. Pruski (MO),        Pruski, J.                 1
!K. Burt-Utley (NOLS),  Burt-Utley, K.             1
!L. Ferrufino (B),      Ferrufino, L.              1
                                                ... 
Ávila                   Ávila                      3
Ávila, A.               Ávila, A.                  1
                        Ávila, F.                 10
Ávila, Andrés           Ávila, F.                  2
Ávila, F.               Ávila, F.                127
Length: 1884, dtype: int64

In [21]:
ori.groupby(['Referencia bibligráfica']).size().reset_index().sort_values(0)

,Referencia bibligráfica,0
168,"de Souza-Secco, R. (2004). Alchorneae (Euphorb...",1
105,"Mora-Osejo, L. E. (1978). Nuevas especies de G...",1
24,"Core, E. L. (1948). The genus Scleria in Colom...",1
55,"Forero-González, E., Piñeros, Z., & Barbosa, C...",1
27,"Cuatrecasas, J. M. (1940). Estudios sobre plan...",1
...,...,...
64,"Garzón de Pérez, M. R. (1993). Flora del Páram...",555
90,"Londoño-Botero, R., Barrera-Lobatón, S., Franc...",559
26,"Cortés-Sánchez, S. P., Hammen, T. van der, & R...",645
121,"Pedraza-Peñalosa, P., Betancur, J., & Franco-R...",1377


-------

In [ ]:
# Multispecies hybrids

taxa_map.loc[taxa_map.Name_submitted.str.contains('×') & 
	taxa_map.Name_matched.notna() & 
	~taxa_map.Name_matched.str.contains(r'×',na=False)]

-------

In [ ]:
df = pd.read_sql(sql='SELECT * FROM Taxa', con=connection)

In [ ]:
df

In [ ]:
#connection.execute(text('SET FOREIGN_KEY_CHECKS = 1'))

In [12]:
connection.close()